<a href="https://colab.research.google.com/github/donghyundavidchoi/Data_Analysis/blob/main/%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%A0%95%EC%A0%9C/%EA%B2%B0%EC%B8%A1%EC%B9%98%EC%A0%9C%EA%B1%B0_groupby_%EA%B7%B8%EB%A3%B9%ED%95%A9%EA%B3%84%EA%B5%AC%ED%95%98%EA%B8%B0_multiindex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# basic1.csv에서 'f1'컬럼 결측 데이터를 제거하고, 'city'와 'f2'을 기준으로 묶어 합계를 구하고, 'city가 경기이면서 f2가 0'인 조건에 만족하는 f1 값을 구하시오.

In [1]:
!gdown --fuzzy https://drive.google.com/file/d/1txRVXp-Eh222PzIeTQ-fuAoUYFR9f7q-/view?usp=sharing
!mkdir -p ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Downloading...
From: https://drive.google.com/uc?id=1txRVXp-Eh222PzIeTQ-fuAoUYFR9f7q-
To: /content/kaggle.json
100% 73.0/73.0 [00:00<00:00, 77.0kB/s]


In [2]:
!kaggle datasets download -d agileteam/bigdatacertificationkr

  0% 0.00/15.5k [00:00<?, ?B/s]
100% 15.5k/15.5k [00:00<00:00, 11.6MB/s]


In [3]:
!unzip /content/bigdatacertificationkr.zip

Archive:  /content/bigdatacertificationkr.zip
  inflating: basic1.csv              
  inflating: basic1_json.json        
  inflating: basic1_xml.xml          
  inflating: basic2.csv              
  inflating: basic3.csv              


In [4]:
import pandas as pd
import numpy as np

In [12]:
data = pd.read_csv('/content/basic1.csv')

In [13]:
data

,id,age,city,f1,f2,f3,f4,f5
0,id01,2.0,서울,NaN,0,NaN,ENFJ,91.297791
1,id02,9.0,서울,70.0,1,NaN,ENFJ,60.339826
2,id03,27.0,서울,61.0,1,NaN,ISTJ,17.252986
3,id04,75.0,서울,NaN,2,NaN,INFP,52.667078
4,id05,24.0,서울,85.0,2,NaN,ISFJ,29.269869
...,...,...,...,...,...,...,...,...
95,id96,92.0,경기,53.0,1,NaN,ENTJ,52.667078
96,id97,100.0,경기,NaN,0,NaN,INFP,67.886373
97,id98,39.0,경기,58.0,2,NaN,INFP,98.429899
98,id99,1.0,경기,47.0,0,NaN,ESFJ,97.381034


---------

# isnull/isna로 결측치 확인

In [14]:
data.isnull().sum()

id       0
age      0
city     0
f1      31
f2       0
f3      95
f4       0
f5       0
dtype: int64

---------

# dropna을 확용한 결측치 제거
Default : DataFrame.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)


In [17]:
data = data.dropna(axis=0, subset=['f1'])

----------

# 결측치가 제대로 제거 됐는지 확인

In [18]:
data.isnull().sum()

id       0
age      0
city     0
f1       0
f2       0
f3      65
f4       0
f5       0
dtype: int64

----------

# groupby를 활용하여 city와 f2를 기준으로 컬럼의 합 구하기

In [23]:
data_groupby_city_f2 = data.groupby(['city','f2']).sum()

In [24]:
data_groupby_city_f2

age     f1          f5
city f2                          
경기   0   720.4  833.0  943.944823
     1   696.0  670.0  657.241212
     2   239.0  311.0  362.300060
대구   0   387.0  527.0  183.199568
     1   217.6  235.0  241.333824
     2   140.0  211.0   79.667919
부산   0   331.0  389.0  284.371097
     1   188.7  315.0  299.270973
     2   -13.5   47.0   67.886373
서울   0   145.0  278.0  218.528577
     1   315.3  534.0  438.485010
     2    68.3  207.0  126.661135

---------

# 경기와 0에 해당하는 f1 값 출력

In [57]:
print(data_groupby_city_f2['f1'].loc['경기', 0])

833.0


.loc[['경기', 0]] 표기는 인덱스를 리스트 형태로 받는 것이라 오류 발생
아래와 같은 형태는 가능

In [66]:
print(data_groupby_city_f2['f1'].loc['경기', [0, 1, 2]])

city  f2
경기    0     833.0
      1     670.0
      2     311.0
Name: f1, dtype: float64


.loc[2, '경기'])의 형태는 오류 발생

-------------

# 멀티인덱스의 인덱스 선택 순서가 groupby 인덱스 순서에 영향을 받을 것이라고 가설 설정
# groupby 컬럼 순서를 바꿔서 groupby 실행후 .loc[2, '경기']) 출력

In [42]:
data_groupby_f2_city = data.groupby(['f2','city']).sum()

In [43]:
data_groupby_f2_city

age     f1          f5
f2 city                          
0  경기    720.4  833.0  943.944823
   대구    387.0  527.0  183.199568
   부산    331.0  389.0  284.371097
   서울    145.0  278.0  218.528577
1  경기    696.0  670.0  657.241212
   대구    217.6  235.0  241.333824
   부산    188.7  315.0  299.270973
   서울    315.3  534.0  438.485010
2  경기    239.0  311.0  362.300060
   대구    140.0  211.0   79.667919
   부산    -13.5   47.0   67.886373
   서울     68.3  207.0  126.661135

In [69]:
print(data_groupby_f2_city['f1'].loc[2, '경기'])

311.0


예상대로 정상 출력   
반대로 .loc['경기', 2]는 오류 발생   
멀티인덱스 환경에서 인덱스 선택은 반드시 앞선 범주부터 선택해야함.

------------

# groupby 없이 결과값을 출력하기 위해 변수 선택 후 sum 실행

In [33]:
print(data['f1'][(data['city'] == '경기') & (data['f2'] == 0)].sum())

833.0
